In [54]:
import json
import folium
import requests
import numpy as np
import pandas as pd
import urllib.request
import matplotlib.cm as cm
from bs4 import BeautifulSoup
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize


In [42]:
#Get geospacial data
GeoData = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

#Get postcodes data
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)

table_contents=[]
soup = BeautifulSoup(page, "lxml")

table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


PostalCodes_df =pd.DataFrame(table_contents)
PostalCodes_df['Borough']=PostalCodes_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

PostalCodes_df = PostalCodes_df.reindex(columns=['PostalCode','Borough','Neighborhood'])

#Merge post codes data with geospacial data
GeoData.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
PostalCodes_df = PostalCodes_df.merge(GeoData, on = 'PostalCode')

#Toronto only neighborhoods
Toronto_Neighborhoods_df = PostalCodes_df[PostalCodes_df['Borough'].str.contains('Toronto')]
Toronto_Neighborhoods_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [26]:
CLIENT_ID = input('Please enter your Foursquare client id')
CLIENT_SECRET = input('Please enter your Foursquare client secret')
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500


In [43]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [45]:
downtown_toronto_data = Toronto_Neighborhoods_df[Toronto_Neighborhoods_df['Borough']=='Downtown Toronto']

downtown_toronto_Venues = getNearbyVenues(names= downtown_toronto_data['Neighborhood'],
                                   latitudes= downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude']
                                  )
downtown_toronto_Venues

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [47]:
# one hot encoding
downtown_toronto_Venues_onehot = pd.get_dummies(downtown_toronto_Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_Venues_onehot['Neighborhood'] = downtown_toronto_Venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_Venues_onehot.columns[-1]] + list(downtown_toronto_Venues_onehot.columns[:-1])
downtown_toronto_Venues_onehot = downtown_toronto_Venues_onehot[fixed_columns]

downtown_toronto_Venues_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
downtown_toronto_Venues_grouped = downtown_toronto_Venues_onehot.groupby('Neighborhood').mean().reset_index()
downtown_toronto_Venues_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.0000,0.00,0.00,0.00,0.017857,0.000000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,...,0.000000,0.0000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.015152,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.0000,0.00,0.00,0.00,0.015152,0.000000,0.000000,0.000000,0.000000
3,Christie,0.000000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.0000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.025000,0.0125,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012500,...,0.012500,0.0125,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,...,0.000000,0.0000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
6,"First Canadian Place, Underground city",0.000000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,...,0.010000,0.0000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000
7,"Garden District, Ryerson",0.000000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.020000,0.0000,0.01,0.00,0.00,0.000000,0.010000,0.010000,0.000000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.010000,0.0000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.0000,0.00,0.00,0.00,0.045455,0.000000,0.045455,0.015152,0.000000


In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_Venues_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_Venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_Venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Bakery,Coffee Shop,Pharmacy,Farmers Market,Seafood Restaurant,Cheese Shop,Restaurant,Cocktail Bar,Beer Bar,Museum
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina,Bar,Boat or Ferry
2,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Restaurant,Burger Joint,Japanese Restaurant,Bubble Tea Shop,Donut Shop
3,Christie,Grocery Store,Café,Park,Candy Store,Nightclub,Athletics & Sports,Italian Restaurant,Restaurant,Baby Store,Coffee Shop
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Hotel,Mediterranean Restaurant,Men's Store,Fast Food Restaurant


In [51]:
# set number of clusters
kclusters = 5

downtown_toronto_Venues_grouped_clustering = downtown_toronto_Venues_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_Venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 3, 0, 0, 0, 0, 0, 4])

In [52]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_toronto_merged = downtown_toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Theater,Café,Event Space,Chocolate Shop,Cosmetics Shop
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Hotel,Japanese Restaurant,Café,Cosmetics Shop,Bubble Tea Shop,Ramen Restaurant,Pizza Place,Diner
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Clothing Store,Gastropub,Cocktail Bar,Cosmetics Shop,Gym,Seafood Restaurant,Department Store
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Bakery,Coffee Shop,Pharmacy,Farmers Market,Seafood Restaurant,Cheese Shop,Restaurant,Cocktail Bar,Beer Bar,Museum
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Restaurant,Burger Joint,Japanese Restaurant,Bubble Tea Shop,Donut Shop


In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [56]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Theater,Café,Event Space,Chocolate Shop,Cosmetics Shop
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Hotel,Japanese Restaurant,Café,Cosmetics Shop,Bubble Tea Shop,Ramen Restaurant,Pizza Place,Diner
15,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Clothing Store,Gastropub,Cocktail Bar,Cosmetics Shop,Gym,Seafood Restaurant,Department Store
20,Downtown Toronto,0,Bakery,Coffee Shop,Pharmacy,Farmers Market,Seafood Restaurant,Cheese Shop,Restaurant,Cocktail Bar,Beer Bar,Museum
24,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Restaurant,Burger Joint,Japanese Restaurant,Bubble Tea Shop,Donut Shop
30,Downtown Toronto,0,Coffee Shop,Café,Clothing Store,Restaurant,Thai Restaurant,Gym,Hotel,Concert Hall,Salad Place,Bakery
36,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Scenic Lookout,Brewery,Restaurant,Fried Chicken Joint,Park,Sporting Goods Shop
42,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,Japanese Restaurant,Bakery,Salad Place,Concert Hall,Asian Restaurant
48,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel,Gym,Deli / Bodega,Japanese Restaurant,Bakery,Seafood Restaurant,American Restaurant
96,Downtown Toronto,0,Café,Pizza Place,Coffee Shop,Pub,Bakery,Park,Italian Restaurant,Restaurant,Beer Store,Liquor Store


In [60]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Downtown Toronto,1,Park,Playground,Trail,Museum,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


In [59]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
87,Downtown Toronto,2,Airport Service,Airport Lounge,Airport Terminal,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina,Bar,Boat or Ferry


In [58]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Downtown Toronto,3,Grocery Store,Café,Park,Candy Store,Nightclub,Athletics & Sports,Italian Restaurant,Restaurant,Baby Store,Coffee Shop


In [57]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
80,Downtown Toronto,4,Café,Bookstore,Bar,Japanese Restaurant,Sandwich Place,Bakery,Nightclub,Beer Bar,Beer Store,French Restaurant
84,Downtown Toronto,4,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Gaming Cafe,Bakery,Burger Joint,Arts & Crafts Store,Mexican Restaurant
